# EddyPro API

This notebook gives an outline for working with the API.

* First step is to define your input primary parameters.  You need to give a siteID, everything else is *optional*.

    * kwargs (dict of all optional settings):\
        * sourceDir: if not provided, assumes data are already in Outputs folder
        * dateRange (if not provided, will run all files in the current year)
        * fileType (defaults to .ghg) must specify otherwise for 
        * metadataTemplate (only required for .dat files)



In [1]:
# Example run for BB on one day in 2024
siteID = 'BB'
kwargs = {
    'dateRange':['2024-04-01','2024-04-02'],
    'sourceDir':"C:/highfreq/BB"
}

# # Example run for All BBS data
siteID = 'BBS'
kwargs = {
    'dateRange':['2023-06-01','2024-05-31'],
    # 'sourceDir':"X:/BBS/EC_Station/2024/20240516/",
    'searchTag':"BBS.FLUX",
    'timeShift':30,
    'fileType':'dat',
    'metaDataTemplate':'C:/highfreq/BBS/TOA5_BBS.FLUX_2023_06_14_1500.metadata',
}


# Cospectral correction options
hf_meth = {
    'Moncrieff et al. (1997)':1,
    'Horst (1997)':2,
    'Ibrom et al. (2007)':3,
    'Fratini et al. (2012)':4, # Recommended for closed path systems, when sufficient data (>1 month) are available
    'Massman (2000, 2001)':5
}

if siteID == 'BBS':
    userDefinedEddyProSettings = {
        'Project':{
            'hf_meth':'5'
            },
        'RawProcess_Settings':{
            'v_offset':0.08,
            'u_offset':-0.04
            },
        }
    kwargs['userDefinedEddyProSettings'] = userDefinedEddyProSettings

if 'dateRange' not in kwargs:
    kwargs['dateRange'] = None


In [2]:
import eddyProAPI
import importlib
importlib.reload(eddyProAPI)
kwargs['queryBiometDatabase'] = True
eP = eddyProAPI.eddyProAPI(siteID,**kwargs)
eP.config['BiometUser']
# eP().preProcessing()
# pre = eddyProAPI.preProcessing(siteID,**kwargs)

{'dbase_metadata': {'timestamp': {'name': 'clean_tv', 'dtype': 'float64', 'precision': 8, 'base': 719529, 'base_unit': 'D', 'resolution': '30min'}, 'traces': {'dtype': 'float32', 'precision': 4}}, 'stage': {'Second': 'Clean/SecondStage', 'Third': 'Clean/ThirdStage', 'Third_Default': 'Clean/ThirdStage_Default_Ustar', 'Third_Advanced': 'Clean/ThirdStage_Full_Ustar_Uncertainty', 'Met': 'Met', 'Flux': 'Flux', 'Manual': 'Met/Manual'}, 'rootDir': {'Database': 'C:/Database/', 'Outputs': 'C:/temp/', 'Datadump': 'X:/'}, 'tasks': {'biometData': {'formatting': {'units_in_header': True, 'na_value': -9999, 'time_vectors': {'timestamp': {'output_name': 'TIMESTAMP_1', 'fmt': '%Y-%m-%d %H%M', 'units': 'yyyy-mm-dd HHMM'}}}, 'traces': {'LW_IN_1_1_1': {'units': 'W+1m-2', 'output_name': 'LWIN_1_1_1'}, 'PA_1_1_1': {'units': 'kPa', 'output_name': 'Pa_1_1_1'}, 'PPFD_IN_1_1_1': {'units': 'umol+1m-2s-1', 'output_name': 'PPFD_1_1_1'}, 'RH_1_1_1': {'units': '%', 'output_name': 'RH_1_1_1'}, 'SW_IN_1_1_1': {'units

{'Biomet.Net': 'C:/Biomet.Net/', 'Database': 'C:/Database/'}

In [7]:
import pandas as pd
Range_index = pd.DatetimeIndex(['2024-05-05','2024-05-06'])
Range_index = pd.DatetimeIndex(Range_index)
Range_index

DatetimeIndex(['2024-05-05', '2024-05-06'], dtype='datetime64[ns]', freq=None)

## Create Optional Auxillary Input Data

* It's best practice to give EddyPro Biomet data (temperature, radiation, etc.) and any dynamic metadata (canopy height) in a .csv file.  Biomet.Net users can create the files from database traces using the code below.  You can also create these files yourself

    * Biomet files: https://www.licor.com/env/support/EddyPro/topics/biomet-data-format.html
    * Dynamic metadata files: https://www.licor.com/env/support/EddyPro/topics/dynamic-metadata.html

In [4]:
# import importlib
# import HelperFunctions 
# importlib.reload(HelperFunctions)
# kwargs = HelperFunctions.queryBiometDatabase(BiometPath = 'C:/Biomet.net/Python/',siteID=siteID,**kwargs)
# # import sys
# print(sys.path)

{'dbase_metadata': {'timestamp': {'name': 'clean_tv', 'dtype': 'float64', 'precision': 8, 'base': 719529, 'base_unit': 'D', 'resolution': '30min'}, 'traces': {'dtype': 'float32', 'precision': 4}}, 'stage': {'Second': 'Clean/SecondStage', 'Third': 'Clean/ThirdStage', 'Third_Default': 'Clean/ThirdStage_Default_Ustar', 'Third_Advanced': 'Clean/ThirdStage_Full_Ustar_Uncertainty', 'Met': 'Met', 'Flux': 'Flux', 'Manual': 'Met/Manual'}, 'rootDir': {'Database': 'C:/Database/', 'Outputs': 'C:/temp/', 'Datadump': 'X:/'}, 'tasks': {'biometData': {'formatting': {'units_in_header': True, 'na_value': -9999, 'time_vectors': {'timestamp': {'output_name': 'TIMESTAMP_1', 'fmt': '%Y-%m-%d %H%M', 'units': 'yyyy-mm-dd HHMM'}}}, 'traces': {'LW_IN_1_1_1': {'units': 'W+1m-2', 'output_name': 'LWIN_1_1_1'}, 'PA_1_1_1': {'units': 'kPa', 'output_name': 'Pa_1_1_1'}, 'PPFD_IN_1_1_1': {'units': 'umol+1m-2s-1', 'output_name': 'PPFD_1_1_1'}, 'RH_1_1_1': {'units': '%', 'output_name': 'RH_1_1_1'}, 'SW_IN_1_1_1': {'units

In [20]:
# Create biomet and dynamicMetadata.csv files
import os
import sys
import importlib

BiometNet = 'C:/Biomet.net/Python/'

# UBC Micromet users can 
if sys.path[0]!=BiometNet:
    sys.path.insert(0,BiometNet)

wd = [p for p in sys.path if p != BiometNet][0]

import csvFromBinary as cfb
importlib.reload(cfb)

kwargs['outputPath']

createAuxilaryData = os.path.abspath(wd+'/config_files/EP_auxillary_data_defs.yml')
auxilaryDpaths=cfb.makeCSV(siteID,tasks=createAuxilaryData,stage='Second',**kwargs)
for key,value in auxilaryDpaths.items():
    kwargs[key] = value

KeyError: 'outputPath'

In [5]:

import eddyProAPI
import importlib
importlib.reload(eddyProAPI)


eP = eddyProAPI.runEP(siteID,**kwargs)

Creating c:\MM_Py\EddyPro_API\temp\group_1_rp_A.eddypro for 1115 files
9
Creating c:\MM_Py\EddyPro_API\temp\group_1_rp_B.eddypro for 1114 files
9
Creating c:\MM_Py\EddyPro_API\temp\group_1_rp_C.eddypro for 1114 files
9
Creating c:\MM_Py\EddyPro_API\temp\group_1_rp_D.eddypro for 1115 files
9
Creating c:\MM_Py\EddyPro_API\temp\group_1_fcc.eddypro for 4458 files
9
Initiating EddyPro Runs for group 1 on 4 cores at normal priority
['c:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_A.eddypro', 'c:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_B.eddypro', 'c:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_C.eddypro', 'c:\\MM_Py\\EddyPro_API\\temp\\group_1_rp_D.eddypro']
[████████████████████████████████████████████████████████████] 4/4

Creating c:\MM_Py\EddyPro_API\temp\group_2_rp_@.eddypro for 1 files
9
Creating c:\MM_Py\EddyPro_API\temp\group_2_fcc.eddypro for 1 files
9
Initiating EddyPro Runs for group 2 on 4 cores at normal priority
['c:\\MM_Py\\EddyPro_API\\temp\\group_2_rp_@.eddypro']
[████████████████████████

KeyError: "None of [('datetime',)] are in the columns"